In [43]:
import pandas as pd 
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# !pip install plotly
import plotly.express as px

In [44]:
#Standardize

data = pd.read_csv('krankenkasse-mit-pbf.csv', sep = ';', names = ['age', 'sex', 'bmi', 'children', 'smoker', 'canton', 'pbf', 'charges'])
data = data.iloc[1:]

data['charges'] = data['charges'].str.replace('’', '').apply(pd.to_numeric)
data['pbf'] = data['pbf'].apply(pd.to_numeric)
data['bmi'] = data['bmi'].apply(pd.to_numeric)
data['age'] = data['age'].apply(pd.to_numeric)
data['children'] = data['children'].apply(pd.to_numeric)

# Encode categorical variables.

data['sex'] = pd.factorize(data['sex'])[0]
data['smoker'] = pd.factorize(data['smoker'])[0]
data['canton'] = pd.factorize(data['canton'])[0]

# Remove encoded categorical variables.

# 1 record with negative percentage of body fat.
data = data[data['pbf']>0]

# removing clients under 21 with 5 children
data = data[~((data['age'] <= 21) & (data['children'] == 5))]

data.head()

,age,sex,bmi,children,smoker,canton,pbf,charges
1,19,0,24.720,0,0,0,35.94,4253
2,18,1,29.416,1,1,1,26.86,2494
3,28,1,28.800,3,1,1,26.92,3138
4,33,1,20.564,0,1,2,7.98,1553
5,32,1,25.504,0,1,2,21.84,2768


In [45]:
features = ['age', 'bmi', 'pbf', 'children', 'smoker', 'canton']

x = StandardScaler().fit_transform(data[features])
y = StandardScaler().fit_transform(data[['charges']])

pca = PCA().fit(x, y)

# number of components, sorted by variance ratio
n_pcs= pca.components_.shape[0]
most_important = [np.abs(pca.components_[i]).argmax() for i in range(n_pcs)]
most_important_names = [features[most_important[i]] for i in range(n_pcs)]

pca_results = pd.DataFrame([[i+1, 
                             pca.explained_variance_ratio_[i], 
                             most_important_names[i]] for i in range(n_pcs)],
                           columns = ['pc', 'variance_ratio', 'feature'])

# cummulative sum used to find out how many components we need to explain 95% of the variance
pca_results['variance_ratio_cumsum'] = pca_results['variance_ratio'].cumsum()
pca_results

,pc,variance_ratio,feature,variance_ratio_cumsum
0,1,0.294015,bmi,0.294015
1,2,0.173517,children,0.467533
2,3,0.168910,smoker,0.636443
3,4,0.161335,canton,0.797779
4,5,0.151148,age,0.948926
5,6,0.051074,bmi,1.000000


We can explain 95% of variance by using the attributes: bmi, children, smoker y-n, canton and age.

In [46]:
# Plot cumulative variance to visualize which attributes to cut off, use only the remaining features as predictors

relevant_features = ['bmi', 'children', 'smoker', 'canton', 'age']

fig = px.line(pca_results, x='pc', y='variance_ratio_cumsum', text='feature',
              title='The number of components needed to explain variance')

fig.add_hline(y=0.95, line_color='red', line_dash='dot', annotation_text='95% cut-off')
fig.update_xaxes(range=[0, len(pca_results)+1], title_text='Number of components')
fig.update_yaxes(range=[0, 1.1], title_text='Cumulative variance (%)')

fig.update_traces(textposition='top left')

fig.show()

In [47]:
from sklearn.cluster import KMeans

# by reducing the number of features with pca we improve the performance of K-means
pca = PCA(len(relevant_features)).fit(x,y)
scores_pca = pca.transform(x)

wcss = []
for i in range(1, 21):
    kmeans_pca = KMeans(n_clusters=i, init='k-means++', random_state=42)
    kmeans_pca.fit(scores_pca)
    wcss.append((i, kmeans_pca.inertia_))
    
kmeans_scores = pd.DataFrame(wcss, columns=['n_clusters', 'wcss'])

fig = px.line(kmeans_scores, x='n_clusters', y='wcss',
             title='K-means with PCA clustering')
fig.add_vline(x=3, line_color='red', line_dash='dot')
fig.show()

C:\Users\kfatur\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=6.



In [48]:
# elbow in the graph above indicates that 3 clusters are needed - 3 groups of clients
N_CLUSTERS = 3

model = KMeans(n_clusters=N_CLUSTERS, init='k-means++', random_state=42)
model.fit(x, y)

data['segment'] = model.labels_

fig = px.scatter(data, y='charges', color='segment')
fig.show()

n_pcs= pca.components_.shape[0]
most_important = [np.abs(pca.components_[i]).argmax() for i in range(n_pcs)]
most_important_names = [features[most_important[i]] for i in range(n_pcs)]

pca_results = pd.DataFrame([[i+1, 
                             pca.explained_variance_ratio_[i], 
                             most_important_names[i]] for i in range(n_pcs)],
                           columns = ['pc', 'variance_ratio', 'feature'])

pca_results['variance_ratio_cumsum'] = pca_results['variance_ratio'].cumsum()
pca_results

,pc,variance_ratio,feature,variance_ratio_cumsum
0,1,0.294015,bmi,0.294015
1,2,0.173517,children,0.467533
2,3,0.168910,smoker,0.636443
3,4,0.161335,canton,0.797779
4,5,0.151148,age,0.948926


In [49]:
data.head()

,age,sex,bmi,children,smoker,canton,pbf,charges,segment
1,19,0,24.720,0,0,0,35.94,4253,1
2,18,1,29.416,1,1,1,26.86,2494,0
3,28,1,28.800,3,1,1,26.92,3138,0
4,33,1,20.564,0,1,2,7.98,1553,0
5,32,1,25.504,0,1,2,21.84,2768,0


In [50]:
# For each of these client groups we calculate the levels for the feature 'bmi'
# Group by segment and calculate the mean bmi per segment
segments = data.groupby('segment').mean()['bmi'].sort_values().to_numpy()

# Calculate the mid point between segment means to get a border between the segments
segments_borders = (segments[1:] + segments[:-1])/2

# lower border, min value of all bmi values
segments_borders = np.insert(segments_borders, 0, data['bmi'].min())
# upper border, max value of all bmi values
segments_borders = np.append(segments_borders, data['bmi'].max())

segments_labels = [x for x in range(len(segments_borders)-1)]

# Cut data with segment borders
data['bmi_group'] = pd.cut(data['bmi'], segments_borders, labels=segments_labels, include_lowest=True)

fig = px.box(data, x='bmi_group', y='charges')
fig.update_yaxes(range=[0, 15000])
fig.show()

# Borders for 0, 1 and 2 level of 'bmi'
segments_borders

array([15.168     , 25.25341639, 28.82683667, 44.904     ])

#### BMI  
level 0: from 15-25.2 bmi  
level 1: 25.3-28.8
level 2: 28.9-45

In [51]:
fig = px.bar(data.groupby('children').mean().reset_index(), x='children', y='charges')
fig.show()

From the chart above we notice that having 0-1 child results in similar charges, 2-3 children results in similar charges, and having 4-5 children also results to similar charges.

In [52]:
# number of children as borders between segments
segments_borders = [0,2,4,6]
segments_labels = list(range(len(segments_borders)-1))
data['children_group'] = pd.cut(data['children'], segments_borders, labels=segments_labels, include_lowest=True)

fig = px.box(data, x='children_group', y='charges')
fig.update_yaxes(range=[0, 15000])
fig.show()

In [53]:
fig = px.bar(data.groupby('smoker').mean().reset_index(), x='smoker', y='charges')
fig.show()
# 0 = smoker, 1 = non-smoker

In [54]:
fig = px.bar(data.groupby('canton').mean().reset_index(), x='canton', y='charges')
fig.show()

In [55]:
# Age-charges link, averge per patient
fig = px.scatter(data.groupby('age').mean().reset_index(),  
             x = 'age',
             y = 'charges',
             trendline = 'ols',
             title = 'Age and medical costs')
fig.show()

With regards only to age would be the lr formula for premium: 179.577*age + (-348.268), R^2 being 0.83. The regression model accounts for a lot of the variance (we see that from the data points being mostly close to the regression line).

In [56]:
# Group by segment and calculate the mean age per segment
segments = data.groupby('segment').mean()['age'].sort_values().to_numpy()

# Calculate the mid point between segment means to get a border between the age segments
segments_borders = (segments[1:] + segments[:-1])/2

# lower border, min value of all bmi values
segments_borders = np.insert(segments_borders, 0, data['age'].min())
# upper border, max value of all bmi values
segments_borders = np.append(segments_borders, data['age'].max())

segments_labels = [x for x in range(len(segments_borders)-1)]

# Cut data with segment borders
data['age_group'] = pd.cut(data['age'], segments_borders, labels=segments_labels, include_lowest=True)

fig = px.box(data, x='age_group', y='charges')
fig.update_yaxes(range=[0, 15000])
fig.show()

segments_borders

array([18.        , 37.77431751, 40.41592066, 64.        ])

Age segments:

    0 = 18-37.8 y
    1 = 37.9-40.4 y
    2 = 40.5-64 y

In [57]:
# 0.95 of variance - normalize variance to 1
variance_scaling = 1 / pca_results['variance_ratio_cumsum'].max()

# Choose the relevant feature and multiply it with the scaled variance (to obtain 100% of the cost)
bmi_variance_ratio = pca_results[pca_results['feature'] == 'bmi']['variance_ratio'] * variance_scaling
children_variance_ratio = pca_results[pca_results['feature'] == 'children']['variance_ratio'] * variance_scaling
smoker_variance_ratio = pca_results[pca_results['feature'] == 'smoker']['variance_ratio'] * variance_scaling
canton_variance_ratio = pca_results[pca_results['feature'] == 'canton']['variance_ratio'] * variance_scaling
age_variance_ratio = pca_results[pca_results['feature'] == 'age']['variance_ratio'] * variance_scaling

# Calculate mean charges per certain group
bmi_group_charges = data.groupby('bmi_group').agg(bmi_group_charges_mean=('charges', 'mean'))

# Mean charges multiplied by the scaled variance ratio
# To get the amount of charges which is directly connected to 'bmi'
bmi_group_charges['bmi_group_charges_scaled'] = bmi_group_charges['bmi_group_charges_mean'].apply(
    lambda x: x*bmi_variance_ratio)
data = data.join(bmi_group_charges, on='bmi_group')

# Mean charges of "children groups" multiplied by the scaled variance ratio
# To get the amount of charges which is directly connected to the number of children.
children_group_charges = data.groupby('children_group').agg(children_group_charges_mean=('charges', 'mean'))
children_group_charges['children_group_charges_scaled'] = children_group_charges['children_group_charges_mean'].apply(
    lambda x: x*children_variance_ratio)
data = data.join(children_group_charges, on='children_group')

# To obtain the amount of charges linked to smoking:
smoker_charges = data.groupby('smoker').agg(smoker_charges_mean=('charges', 'mean'))
smoker_charges['smoker_charges_scaled'] = smoker_charges['smoker_charges_mean'].apply(lambda x: x*smoker_variance_ratio)
data = data.join(smoker_charges, on='smoker')

# To obtain the amount of charges linked to canton of residence:
canton_charges = data.groupby('canton').agg(canton_charges_mean=('charges', 'mean'))
canton_charges['canton_charges_scaled'] = canton_charges['canton_charges_mean'].apply(lambda x: x*canton_variance_ratio)
data = data.join(canton_charges, on='canton')

# To obtain the amount linked to age
age_charges = data.groupby('age_group').agg(age_charges_mean=('charges', 'mean'))
age_charges['age_charges_scaled'] = age_charges['age_charges_mean'].apply(lambda x: x*age_variance_ratio)
data = data.join(age_charges, on='age_group')

# A possible way to determine premium, based on categories defined by pca and later k-clustering"
data['premium'] = \
    data['bmi_group_charges_scaled'] \
    + data['children_group_charges_scaled'] \
    + data['smoker_charges_scaled'] \
    + data['canton_charges_scaled'] \
    + data['age_charges_scaled']

data.head()

,age,sex,bmi,children,smoker,canton,pbf,charges,segment,bmi_group,...,bmi_group_charges_scaled,children_group_charges_mean,children_group_charges_scaled,smoker_charges_mean,smoker_charges_scaled,canton_charges_mean,canton_charges_scaled,age_charges_mean,age_charges_scaled,premium
1,19,0,24.720,0,0,0,35.94,4253,1,0,...,1264.508001,6602.383465,1207.289592,12788.890110,2276.440323,6007.180124,1021.334539,4624.869565,736.662305,6506.234759
2,18,1,29.416,1,1,1,26.86,2494,0,2,...,3261.675994,6602.383465,1207.289592,5136.220547,914.254440,7643.046703,1299.462879,4624.869565,736.662305,7419.345209
3,28,1,28.800,3,1,1,26.92,3138,0,1,...,1740.486262,7484.912088,1368.665802,5136.220547,914.254440,7643.046703,1299.462879,4624.869565,736.662305,6059.531688
4,33,1,20.564,0,1,2,7.98,1553,0,0,...,1264.508001,6602.383465,1207.289592,5136.220547,914.254440,6750.307692,1147.680317,4624.869565,736.662305,5270.394655
5,32,1,25.504,0,1,2,21.84,2768,0,1,...,1740.486262,6602.383465,1207.289592,5136.220547,914.254440,6750.307692,1147.680317,4624.869565,736.662305,5746.372916


Possible approach: We sort each client according to the most important categories in each of the crucial features (bmi: 3 categories, smoking: 2 cat., children: 3 cat., canton: 4 cat.)

### Overview of the (absolute) components of a premium

Each client is sorted according to the 4 main features (bmi, children, smoking, canton) and assigned a premium cost accordingly. E.g. If somebody is in bmi group 0, has 2 children (group 1), is a smoker (group 0), lives in canton 3 and is 40 years old (group 1), the amounts making up the premium are:  
1504 + 1628 + 2708 + 1271 + 896 = 8007

In [58]:
data.groupby(['bmi_group']).first()['bmi_group_charges_scaled'].reset_index()

,bmi_group,bmi_group_charges_scaled
0,0,1264.508001
1,1,1740.486262
2,2,3261.675994


In [59]:
data.groupby(['children_group']).first()['children_group_charges_scaled'].reset_index()

,children_group,children_group_charges_scaled
0,0,1207.289592
1,1,1368.665802
2,2,874.456977


In [60]:
data.groupby(['smoker']).first()['smoker_charges_scaled'].reset_index()

,smoker,smoker_charges_scaled
0,0,2276.440323
1,1,914.254440


In [61]:
data.groupby(['canton']).first()['canton_charges_scaled'].reset_index()

,canton,canton_charges_scaled
0,0,1021.334539
1,1,1299.462879
2,2,1147.680317
3,3,1068.890801


In [62]:
data.groupby(['age_group']).first()['age_charges_scaled'].reset_index()

,age_group,age_charges_scaled
0,0,736.662305
1,1,895.534169
2,2,1410.680493


In [63]:
#Did we account for all charges? 'charges' are taken from the dataset, 'premium' is taken from our calculation. 
print('Difference between charges and premiums:', data['charges'].sum() - data['premium'].sum())

Difference between charges and premiums: 0.0
